# Gemini API: OAuth 快速入門


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/main/quickstarts/Authentication_with_OAuth.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 上執行</a>
  </td>
</table>


Gemini API 的某些部分，例如模型調校和語義檢索，使用 OAuth 進行驗證。

如果是初學者，你應該從使用 [API 金鑰](https://github.com/doggy8088/gemini-api-cookbook/blob/main/quickstarts/Authentication.zh.ipynb) 開始，只有在你需要這些功能時，才回到此 OAuth 指南。

為了幫助你開始使用 OAuth，此筆記本展示了一個簡化的做法，適用于測試環境。

對於生產環境，請在 [選擇存取憑證](https://developers.google.com/workspace/guides/create-credentials#choose_the_access_credential_that_is_right_for_you) 之前瞭解關於 [驗證和授權](https://developers.google.com/workspace/guides/auth-overview) 的資訊，該憑證適合你的應用程式。


## 先決條件

要執行此快速入門，你需要：

* 你的本機電腦已安裝 [Google Cloud CLI](https://cloud.google.com/sdk/docs/install-sdk)。
* [一個 Google Cloud 專案](https://developers.google.com/workspace/guides/create-project)。

如果你已在 Google AI Studio 中建立 API 金鑰，表示 Google Cloud 專案已為你建立了。前往 [Google AI Studio](https://aistudio.google.com/app/apikey) 並記住 Google Cloud 專案名稱，以使用該專案。


## 設定 Cloud 專案

若要完成此快速入門，你首先需要設定你的 Cloud 專案。

### 1. 啟用 API

在使用 Google API 之前，你需要在 Google Cloud 專案中將它們開啟。

*   在 Google Cloud 主控台中，[啟用](https://console.cloud.google.com/flows/enableapi?apiid=generativelanguage.googleapis.com) Google Generative Language API。如果你在 AI Studio 中建立了 API 金鑰，這項動作已為你完成。<br>

### 2. 設定 OAuth 同意畫面

接著，設定專案的 OAuth 同意畫面，並將你自己新增為測試使用者。如果你已為你的 Cloud 專案完成此步驟，請跳至下一區段。

1. 在 Google Cloud 主控台中，前往 [OAuth 同意畫面](https://console.cloud.google.com/apis/credentials/consent)，可在 **選單** > **API 與服務** > **OAuth 同意畫面** 中找到。

2. 為你的應用程式選擇 **外部** 使用者類型，然後按一下 **建立** 。

3. 填寫應用程式註冊表 (你可以將大部分欄位留空)，然後按一下 **儲存並繼續** 。

4. 目前，你可以略過新增範圍，並按一下 **儲存並繼續** 。未來，當你建立應用程式以在 Google Workspace 組織外部使用時，你必須新增並驗證你的應用程式所需的授權範圍。

5. 新增測試使用者：
    1. 在 **測試使用者** 下方，按一下 **新增使用者** 。
    2. 輸入你的電子郵件地址和任何其他已授權的測試使用者，然後按一下 **儲存並繼續** 。

6. 檢閱你的應用程式註冊摘要。若要進行變更，請按一下 **編輯** 。如果應用程式註冊看起來沒問題，請按一下 **返回儀表板** 。

### 3. 為桌面應用程式授權認證資料

若要以最終使用者身分驗證，並在你的應用程式中存取使用者資料，你需要建立一個或多個 OAuth 2.0 識別碼。識別碼用於向 Google 的 OAuth 伺服器識別單一應用程式。如果你的應用程式在多個平台上執行，你必須為每個平台建立獨立的識別碼。

1. 在 Google Cloud 主控台中，前往 [認證資料](https://console.cloud.google.com/apis/credentials/consent)，可在 **選單** > **API 與服務** > **認證資料** 中找到。

2. 按一下 **建立認證資料** > **OAuth 識別碼** 。
3. 按一下 **應用程式類型** > **桌面應用程式** 。
4. 在 **名稱** 欄位中，輸入認證資料的名稱。這個名稱只會顯示在 Google Cloud 主控台中。
5. 按一下 **建立** 。接著會出現建立的 OAuth 識別碼畫面，顯示你的新識別碼和識別碼密鑰。
6. 按一下 **確定** 。新建立的認證資料會顯示在 **OAuth 2.0 識別碼** 下方。
7. 按一下下載按鈕以儲存 JSON 檔案。它會儲存為 `client_secret_<identifier>.json`。


## 設定應用程式的預設認證資訊

在此快速入門中，我們使用 [應用程式的預設認證資訊](https://cloud.google.com/docs/authentication/application-default-credentials) 來驗證身分。


### 將 Client Secret 加入 Colab 祕密中

如果你需要在 Google Colab 中頻繁使用 Gemini API 的 OAuth，最簡單的方法就是將 `client_secret.json` 檔案的內容加入 Colab 的祕密管理器中。

1. 開啟你的 Google Colab 筆記本，然後按一下左側面板中的 🔑 **祕密** 標籤。
2. 建立一個新的祕密，將其命名為 `CLIENT_SECRET`。
3. 在文字編輯器中開啟 `client_secret.json` 檔案，然後將內容複製/貼上到 `CLIENT_SECRET` 的 `Value` 輸入框中。
4. 切換左側的按鈕，以允許筆記本存取祕密。

現在，你可以編寫程式來建立檔案，而不是每次都上傳檔案。在你允許存取後，所有 Google Colab 筆記本都能使用 Client Secret。


In [ ]:
from google.colab import userdata
import pathlib
pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

413

### 設定應用程式預設認證資料

若要將 `client_secret.json` 檔案轉換為可用的認證資料，請將其位置傳遞給 `gcloud auth application-default login` 命令的 `--client-id-file` 引數。

本教學課程中簡化的專案設定會觸發 **Google 尚未驗證此應用程式** 對話方塊。這是正常的，請選擇 **繼續** 。

你將需要對每個新的 Google Colab 記事本或執行時期執行此步驟一次。

**注意事項** ：仔細遵循下列命令列印的指示 (不要只按一下連結)。此外，請確保你的本機 `gcloud --version` 是 [最新版本](https://cloud.google.com/sdk/docs/release-notes)，以符合在 Google Colab 中預先安裝的版本。


In [ ]:
!gcloud auth application-default login \
  --no-browser --client-id-file client_secret.json \
  --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning,https://www.googleapis.com/auth/generative-language.retriever'


你需要的特定「範圍」取決於你使用的 API。例如，檢視 [`tunedModels.create`](https://ai.google.dev/api/rest/v1beta/tunedModels/create#authorization-scopes) API 參考文件：

> 需要以下 OAuth 範圍之一：

> *   `https://www.googleapis.com/auth/generative-language.tuning`

此範例要求所有調整和語意檢索範圍，但最佳實務做法是使用最小的範圍集，以確保安全性並增加使用者信心。


## 使用帶有 OAuth 的 Python SDK

Python SDK 將會自動搜尋並使用應用程式預設憑證。


In [ ]:
!pip install -U -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.7 MB/s eta 0:00:00


讓我們進行一個快速測試。請注意，我們還沒有使用 `genai.configure()` 設定 API 金鑰！


In [ ]:
import google.generativeai as genai

print('Available base models:', [m.name for m in genai.list_models()])

# 附錄


## 從 Colab 進行已驗證的 REST 呼叫

一般而言，我們建議在適用的情況下使用 Python SDK 與 Gemini API 進行互動。此範例說明如何從 Python 進行 OAuth 驗證的 REST 呼叫以進行偵錯或測試。它假設你已從 Quickstart 設定應用程式的預設憑證。


In [ ]:
import requests

access_token = !gcloud auth application-default print-access-token

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {access_token[0]}',
}

response = requests.get('https://generativelanguage.googleapis.com/v1/models', headers=headers)
response_json = response.json()

# All the model names
for model in response_json['models']:
    print(model['name'])

### 共享調整模型

部分 beta API 功能可能尚未獲得 Python SDK 支援。此範例說明如何從 Python 發出 REST 呼叫，將權限加入調整模型。


In [ ]:
import requests

model_name = ''   # @param {type:"string"}
emailAddress = '' # @param {type:"string"}


access_token = !gcloud auth application-default print-access-token

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {access_token[0]}',
}

body = {
  'granteeType': 'USER',        # Or 'GROUP' or 'EVERYONE' https://ai.google.dev/api/rest/v1beta/tunedModels.permissions
  'emailAddress': emailAddress, # Optional if 'granteeType': 'EVERYONE'
  'role': 'READER'
}

response = requests.post(f'https://generativelanguage.googleapis.com/v1beta/tunedModels/{model_name}/permissions', json=body, headers=headers)
print(response.json())


## 使用服務帳戶進行驗證

Google Cloud [服務帳戶](https://cloud.google.com/iam/docs/service-account-overview) 是不代表人類使用者的帳戶。在人類未直接參與 (像是應用程式呼叫 Gemini API 協助使用者完成某項要求，但並未根據身分驗證使用者) 時，服務帳戶提供方法管理驗證和授權。使用服務帳戶驗證 Gemini API 的一種簡單方式，就是使用 [服務帳戶金鑰](https://cloud.google.com/docs/authentication/provide-credentials-adc#local-key)。

這份指南會簡要說明如何在 Google Colab 中使用服務帳戶金鑰。

**重要資訊：** 服務帳戶金鑰可能存在安全性風險！詳情請參閱 [管理服務帳戶金鑰的最佳做法](https://cloud.google.com/iam/docs/best-practices-for-managing-service-account-keys)。

### 1. 建立服務帳戶

依照說明建立 [服務帳戶](https://cloud.google.com/iam/docs/service-accounts-create#creating)。如果手動執行這些操作，**主控台** 說明最簡單。

### 2. 建立服務帳戶金鑰

依照說明建立 [服務帳戶金鑰]( https://cloud.google.com/iam/docs/keys-create-delete#creating)。記下已下載金鑰的名稱。

### 3. 將服務帳戶金鑰新增至 Colab

1. 開啟 Google Colab 筆記本，然後按一下左側面板中的 🔑 **祕密** 標籤。
2. 建立新祕密，名稱為 `SERVICE_ACCOUNT_KEY`。
3. 在文字編輯器中開啟服務帳戶金鑰文件，然後複製/貼上內容至 `SERVICE_ACCOUNT_KEY` 的 `值` 輸入方塊中。
4. 切換左側的按鈕，以允許筆記本存取祕密。

### 4. 透過服務帳戶金鑰驗證 Python SDK


In [ ]:
import google.generativeai as genai
import pathlib
from google.colab import userdata
from google.oauth2 import service_account

pathlib.Path('service_account_key.json').write_text(userdata.get('SERVICE_ACCOUNT_KEY'))

credentials = service_account.Credentials.from_service_account_file('service_account_key.json')

# Adjust scopes as needed
scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/generative-language.retriever'])

genai.configure(credentials=scoped_credentials)

print('Available base models:', [m.name for m in genai.list_models()])